# Exercise 4
Due:  Tue November 19, 8:00am

## Node2Vec
1. Implement custom dataset that samples pq-walks
    - Use the utility function from torch_cluster that actually performs the walks
2. Implement Node2Vec module and training
	- Node2Vec essentially consists of a torch.Embedding module and a loss function
3. Evaluate node classification performance on Cora
4. Evaluate on Link Prediction: Cora, PPI
    - use different ways to combine the node two embeddings for link prediction

Bonus Question: are the predictions stable wrt to the random seeds of the walks?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch_geometric as pyg
from tqdm import tqdm
import torch_cluster
import sklearn

In [4]:
# find device
if torch.cuda.is_available(): # NVIDIA
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # apple M1/M2
    device = torch.device('mps') 
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [22]:
dataset = pyg.datasets.Planetoid(root='./dataset/cora', name='Cora')
cora = dataset[0]
dataset = pyg.datasets.PPI(root='./dataset/ppi')
ppi = dataset[0]

In [23]:
cora

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [27]:
ppi

Data(x=[1767, 50], edge_index=[2, 32318], y=[1767, 121])

## node2vec embedding training
Here the main training and everything on the graph level is happening.

It might be a good idea to create a dataset of walks (fixed for the whole training process) first to get the whole training process running before attempting to create a train_loader that on-demand samples those walks on-demand.

## Node classification performance
just a small MLP or even linear layer on the embeddings to predict node classes. Accuracy should be above 60%. Please compare your results to those you achieved with GNNs.

In [ ]:
# as the simple MLP is pretty straightforward
model = torch.nn.Sequential(
    torch.nn.Linear(embedding_dim, 256), # Input layer
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128), # Hidden layer 2
    torch.nn.ReLU(),
    torch.nn.Linear(128, cora_dataset.num_classes), # Output layer
)
model = model.to(device)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # define an optimizer
criterion = torch.nn.CrossEntropyLoss()  # define loss function

node2vec_embeddings = embedding.to(device)
cora = cora.to(device)

for epoch in range(100):  # 100 epochs
    model.train()
    optimizer.zero_grad()
    out = model(node2vec_embeddings[cora.train_mask])  # forward pass
    loss = criterion(out, cora.y[cora.train_mask]) 
    loss.backward()  
    optimizer.step()

    # print out loss info
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.3e}")

def get_accuracy(model, embeddings, y, mask):
    out = model(embeddings[mask])
    pred = out.argmax(dim=1)
    acc = sklearn.metrics.accuracy_score(y[mask].cpu().numpy(), pred.cpu().detach().numpy())
    return acc

train_acc = get_accuracy(model, node2vec_embeddings, cora.y, cora.train_mask)
val_acc = get_accuracy(model, node2vec_embeddings, cora.y, cora.val_mask)
test_acc = get_accuracy(model, node2vec_embeddings, cora.y, cora.test_mask)
    
print(f"node classification accuracy for cora: {test_acc:.2f} (train: {train_acc:.2f}, val: {val_acc:.2f})")

## link prediction on trained embeddings
this should only train simple MLPs.

Note: for link prediction to be worthwhile, one needs to train the embeddings on a subset of the graph (less edges, same nodes) instead of the whole graph.

In [32]:
# for link prediction, do something like the following
link_splitter = pyg.transforms.RandomLinkSplit(is_undirected=True)
train_data, val_data, test_data = link_splitter(cora)
train_data
# the positive and negative edges are in "edge_label_index" with "edge_label" 
# indicating whether an edge is a true edge or not.

Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[7392], edge_label_index=[2, 7392])

In [28]:
test_data

Data(x=[2708, 1433], edge_index=[2, 8446], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110])

In [ ]:
# retrain node2vec on train_data

In [33]:
# use those (new) embeddings for link prediction